In [ ]:
import numpy as np
import os

from pyriemann.utils.distance import distance_riemann
from pyriemann.classification import MDM, FgMDM
from machine_learning import TSclassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline, Pipeline


In [ ]:
from loaddata import Dataset_Left_Right_MI
dataset_name = 'Pan2023'

fs = 250
freqband = [8,30]
datapath = r'E:\工作进展\小论文2023会议\数据处理python\datasets'

dataset = Dataset_Left_Right_MI(dataset_name,fs,fmin=freqband[0],fmax=freqband[1],tmin=0,tmax=4,path=datapath)

In [ ]:
n_estimators=50
ABC_algorithm="SAMME"

clf0 = MDM()
clf1 = AdaBoostClassifier(clf0,n_estimators=n_estimators,algorithm=ABC_algorithm)

clf2 = FgMDM()
clf3 = AdaBoostClassifier(clf2,n_estimators=n_estimators,algorithm=ABC_algorithm)

clf4 = TSclassifier(clf=LDA())
clf5 = AdaBoostClassifier(clf4,n_estimators=n_estimators,algorithm=ABC_algorithm)

clf6 = TSclassifier()
clf7 = AdaBoostClassifier(clf6,n_estimators=n_estimators,algorithm=ABC_algorithm)


In [ ]:
from sklearn.model_selection import ShuffleSplit
# cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

In [ ]:
from pyriemann.estimation import Covariances
data, label = dataset.get_data([1])
data = Covariances(estimator='lwf').transform(data)

In [ ]:
# 比较不同分类器的效果

n_jobs = 15

# 1.MDM
scores = cross_val_score(clf0, data, label, cv=cv, n_jobs=n_jobs)
print("MDM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 2.AdaBoost-MDM
scores = cross_val_score(clf1, data, label, cv=cv, n_jobs=n_jobs)
print("Adaboost-MDM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 3.FgMDM
scores = cross_val_score(clf2, data, label, cv=cv, n_jobs=n_jobs)
print("FgMDM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 4.AdaBoost-FgMDM
scores = cross_val_score(clf3, data, label, cv=cv, n_jobs=n_jobs)
print("AdaBoost-FgMDM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 5.TSLDA
scores = cross_val_score(clf4, data, label, cv=cv, n_jobs=n_jobs)
print("TSLDA Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 6.AdaBoost-TSLDA
scores = cross_val_score(clf5, data, label, cv=cv, n_jobs=n_jobs)
print("AdaBoost-TSLDA Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 7.TSGLM
scores = cross_val_score(clf6, data, label, cv=cv, n_jobs=n_jobs)
print("TSGLM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# 8.AdaBoost-TSGLM
scores = cross_val_score(clf7, data, label, cv=cv, n_jobs=n_jobs)
print("AdaBoost-TSGLM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from joblib import Memory

from machine_learning.classifier import svc,lda,lr,knn,dtc,rfc,etc,abc,gbc,gnb,mlp
from machine_learning import RiemannCSP as CSP
from machine_learning import TS
# fee = CSP()
fee = TS()

# 设置缓存目录
cachedir = 'my_cache_directory'
memory = Memory(cachedir, verbose=0)

clf1 = make_pipeline(fee, svc, memory=memory)
scores = cross_val_score(clf1, data, label, cv=cv, n_jobs=15)
print("classifier1 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf2 = make_pipeline(fee, lda, memory=memory)
scores = cross_val_score(clf2, data, label, cv=cv, n_jobs=15)
print("classifier2 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf3 = make_pipeline(fee, lr, memory=memory)
scores = cross_val_score(clf3, data, label, cv=cv, n_jobs=15)
print("classifier3 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf4 = make_pipeline(fee, knn, memory=memory)
scores = cross_val_score(clf4, data, label, cv=cv, n_jobs=15)
print("classifier4 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf5 = make_pipeline(fee, dtc, memory=memory)
scores = cross_val_score(clf5, data, label, cv=cv, n_jobs=15)
print("classifier5 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf6 = make_pipeline(fee, rfc, memory=memory)
scores = cross_val_score(clf6, data, label, cv=cv, n_jobs=15)
print("classifier6 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf7 = make_pipeline(fee, etc, memory=memory)
scores = cross_val_score(clf7, data, label, cv=cv, n_jobs=15)
print("classifier7 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf8 = make_pipeline(fee, abc, memory=memory)
scores = cross_val_score(clf8, data, label, cv=cv, n_jobs=15)
print("classifier8 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf9 = make_pipeline(fee, gbc, memory=memory)
scores = cross_val_score(clf9, data, label, cv=cv, n_jobs=15)
print("classifier9 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf10 = make_pipeline(fee, gnb, memory=memory)
scores = cross_val_score(clf10, data, label, cv=cv, n_jobs=15)
print("classifier10 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf11 = make_pipeline(fee, mlp, memory=memory)
scores = cross_val_score(clf11, data, label, cv=cv, n_jobs=15)
print("classifier12 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from joblib import Memory

from machine_learning.classifier import svc,lda,lr,knn,dtc,rfc,etc,abc,gbc,gnb,mlp
from machine_learning import TS
fee = TS()

# 设置缓存目录
cachedir = 'my_cache_directory'
memory = Memory(cachedir, verbose=0)

clf1 = make_pipeline(fee, svc, memory=memory)
scores = cross_val_score(clf1, data, label, cv=cv, n_jobs=15)
print("classifier1 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf2 = make_pipeline(fee, lda, memory=memory)
scores = cross_val_score(clf2, data, label, cv=cv, n_jobs=15)
print("classifier2 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf3 = make_pipeline(fee, lr, memory=memory)
scores = cross_val_score(clf3, data, label, cv=cv, n_jobs=15)
print("classifier3 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf4 = make_pipeline(fee, knn, memory=memory)
scores = cross_val_score(clf4, data, label, cv=cv, n_jobs=15)
print("classifier4 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf5 = make_pipeline(fee, dtc, memory=memory)
scores = cross_val_score(clf5, data, label, cv=cv, n_jobs=15)
print("classifier5 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf6 = make_pipeline(fee, rfc, memory=memory)
scores = cross_val_score(clf6, data, label, cv=cv, n_jobs=15)
print("classifier6 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf7 = make_pipeline(fee, etc, memory=memory)
scores = cross_val_score(clf7, data, label, cv=cv, n_jobs=15)
print("classifier7 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf8 = make_pipeline(fee, abc, memory=memory)
scores = cross_val_score(clf8, data, label, cv=cv, n_jobs=15)
print("classifier8 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf9 = make_pipeline(fee, gbc, memory=memory)
scores = cross_val_score(clf9, data, label, cv=cv, n_jobs=15)
print("classifier9 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf10 = make_pipeline(fee, gnb, memory=memory)
scores = cross_val_score(clf10, data, label, cv=cv, n_jobs=15)
print("classifier10 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf11 = make_pipeline(fee, mlp, memory=memory)
scores = cross_val_score(clf11, data, label, cv=cv, n_jobs=15)
print("classifier12 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN  # K 近邻
from sklearn.tree import DecisionTreeClassifier as DTC  # 决策树
from sklearn.ensemble import RandomForestClassifier as RFC  # 随机森林
from sklearn.ensemble import ExtraTreesClassifier as ETC  # 极端随机森林
from sklearn.model_selection import GridSearchCV

knn = KNN(n_neighbors=3)
dtc = DTC(min_samples_split=3)
rfc = RFC(n_estimators=100)
etc = ETC(n_estimators=100)

clf4 = make_pipeline([('fee', fee),('clf', knn)], memory=memory)
# 创建参数网格
param_grid = {
    'clf__n_neighbors': [3, 5, 7]
}

# 实例化GridSearchCV
grid_search = GridSearchCV(clf4, param_grid=param_grid)
scores = cross_val_score(grid_search, data, label, cv=cv, n_jobs=15)
print("classifier4 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))


scores = cross_val_score(clf4, data, label, cv=cv, n_jobs=15)
print("classifier4 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf5 = make_pipeline(fee, dtc, memory=memory)
scores = cross_val_score(clf5, data, label, cv=cv, n_jobs=15)
print("classifier5 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf6 = make_pipeline(fee, rfc, memory=memory)
scores = cross_val_score(clf6, data, label, cv=cv, n_jobs=15)
print("classifier6 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

clf7 = make_pipeline(fee, etc, memory=memory)
scores = cross_val_score(clf7, data, label, cv=cv, n_jobs=15)
print("classifier7 Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
from machine_learning import TS
from sklearn.neighbors import KNeighborsClassifier as KNN  # K 近邻
from sklearn.tree import DecisionTreeClassifier as DTC  # 决策树
from sklearn.ensemble import RandomForestClassifier as RFC  # 随机森林
from sklearn.ensemble import ExtraTreesClassifier as ETC  # 极端随机森林
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from joblib import Memory
# 设置缓存目录
cachedir = 'my_cache_directory'
memory = Memory(cachedir, verbose=0)

fee = TS()
knn = KNN(n_neighbors=3)
dtc = DTC(min_samples_split=3)
rfc = RFC(n_estimators=100)
etc = ETC(n_estimators=100)

pipe = Pipeline([('fee', fee),('clf', rfc)], memory=memory)
# 创建参数网格
param_grid = {
    'clf__n_estimators': [20, 50, 100],
    'clf__max_depth': [5, 10, 20],
    'clf__min_samples_split': [2, 5, 10]
}

# 实例化GridSearchCV
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, n_jobs=15)
grid_search.fit(data, label)

# 获取所有参数组合的评估结果
cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"{mean_score:.3f} for {params}")



In [ ]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# 假设 'data' 是您的特征数据，'label' 是标签
# 'cv' 是交叉验证策略，例如 KFold 或 StratifiedKFold

# 设置管道和参数网格
knn = KNeighborsClassifier()
pipe = Pipeline([('fee', fee),('clf', knn)], memory=memory)
param_grid = {'clf__n_neighbors': [3, 5, 7]}

# 实例化 GridSearchCV
grid_search = GridSearchCV(pipe, param_grid=param_grid)

# 使用 cross_val_score 来执行交叉验证
# 注意：这里我们设置 return_estimator=True 来返回每个折叠的估计器
cv_results = cross_validate(grid_search, data, label, cv=cv, n_jobs=15, return_estimator=True)

# Access the estimator from each fold
for fold, estimator in enumerate(cv_results['estimator']):
    print(f"Fold {fold}:")
    print(f"Best parameters: {estimator.best_params_}")
    print(f"Best cross-validation score: {estimator.best_score_}")


In [1]:
from transfer_learning.tl_classifier import TL_Classifier
from joblib import Memory

# 设置缓存目录
cachedir = 'my_cache_directory'
memory = Memory(cachedir, verbose=0)

classifier = TL_Classifier(memory=memory)

In [ ]:
from pre_processing